This is an experimental playground for testing typeclass implementation in python. This experiment will rely on the typechecking capabilities in python 3.12, with typechecking done by pylance/pyright.

The approach we use in this experiment is borrowed from the article (Typeclasses in Python)[https://sobolevn.me/2021/06/typeclasses-in-python]. The core idea is to create a wrapper decorator around the singledispatch tool that comes in python's standard functools library.

Now, let's get started!

In [23]:
from functools import singledispatch, update_wrapper
from dataclasses import dataclass

We can use the singledispatch decorator to register different functions and dispatch on type. So this basically already gives us typeclasses. 

In [16]:
# Create a generic function
@singledispatch
def greet(instance) -> str:
    """Default case."""
    raise NotImplementedError

# Register a function for a specific type
@greet.register
def _greet_str(instance: str) -> str:
    return 'Hello, {0}!'.format(instance)

# Custom type
@dataclass
class MyUser(object):
    name: str

# Register a function for a custom type
@greet.register
def _greet_myuser(instance: MyUser) -> str:
    return 'Hello again, {0}'.format(instance.name)

# Use the generic function
print(greet('world'))
print(greet(MyUser(name='example')))

# Fails without a type error if we do not register a function for a type!
# Though it should be noted that we can set a default function.
try:
    print(greet(1))
    assert False, 'This should not happen'
except NotImplementedError:
    print('We get a runtime error if we do not register a function for a type')

Hello, world!
Hello again, example
We get a runtime error if we do not register a function for a type


This approach is almost exactly what we want, but singledispatch has no typechecking! So what we want is to build a wrapper around singledispatch that actually gives us typeclass capabilities.

In [26]:
def method(func):
    dispatcher = singledispatch(func)
    def wrapper(*args, **kwargs):
        return dispatcher(*args, **kwargs)
    
    wrapper.overload = dispatcher.register
    update_wrapper(wrapper, func)

    return wrapper

# Create a typeclass method
@method
def greet(instance) -> str:
    """Default case."""
    raise NotImplementedError

# Register a function for a specific type
@greet.overload
def _greet_str(instance: str) -> str:
    return 'Hello, {0}!'.format(instance)

# Custom type
@dataclass
class MyUser(object):
    name: str

# Register a function for a custom type
@greet.overload
def _greet_myuser(instance: MyUser) -> str:
    return 'Hello again, {0}'.format(instance.name)

# Use the generic function
print(greet('world'))
print(greet(MyUser(name='example')))

x = MyUser

Hello, world!
Hello again, example
Hello again, example
